In [1]:
from langchain.prompts import Prompt
from langchain_core.prompts import PromptTemplate

from function_registry import FunctionRegistry
import os
from langchain_cerebras import ChatCerebras
from dotenv import load_dotenv
# Set up the environment
from langchain.chat_models import init_chat_model

load_dotenv()

llm = ChatCerebras(
    model="llama-3.3-70b",
    api_key=os.getenv("CEREBRAS_API_KEY")
)

In [2]:
from parser import GoogleParser
import ai_functions  # Important for registering functions

registry = FunctionRegistry(docstring_parser=GoogleParser())
registry.register_decorated_functions()
tools = registry.get_tools()
print(f"Tools: {tools}")
print(f"Registered {len(tools)} tools")

Tools: []
Registered 0 tools


In [3]:
# This is a workaround to get the function signature
from langchain.tools import Tool
from pydantic import BaseModel, Field
from functools import partial

# Tools in LangChain-Format bringen
def make_tool_wrapper(fn_name):
    def _run(*args, **kwargs):
        # Positional Dict -> kwargs
        if args and isinstance(args[0], dict):
            kwargs = args[0]
        return registry.call_function(fn_name, **kwargs)

    return _run


# lc_tools = []
# for tool_spec in tools:
#     name = tool_spec["function"]["name"]
#     description = tool_spec["function"]["description"]
#     fn = lambda **kwargs: registry.call_function(name, **kwargs)  # Achtung: capture!
#     lc_tools.append(
#         Tool.from_function(
#             name=name,
#             description=description,
#             func=fn
#         )
#     )
# Generates SQL queries and returns them as a string
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    sql_query: str = Field(description="SQl query to get the answer")

In [4]:
from ai_functions import calculate_m2_for_asset_components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState

query = "Wie gross ist die Fensterfläche im 2. Obergeschoss?"
# agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
# response = agent.invoke("Wie gross ist die Fensterfläche im 2. Obergeschoss?")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful facility management assistant. Your task is to answer questions about the facility."),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{messages}"),
    ]
)
tools = [ResponseFormatter, calculate_m2_for_asset_components, ]
# langgraph_agent_executor = create_react_agent(llm, tools, prompt=prompt)
# messages = langgraph_agent_executor.invoke(
#     {"messages": [("user", query)]},
# )
# print(messages["messages"][-1])

model_with_tools = llm.bind_tools(tools, tool_choice='required')
print(model_with_tools.invoke(query))



content='' additional_kwargs={'tool_calls': [{'id': '49d2c5bec', 'function': {'arguments': '{"answer": "Die Fensterfl\\u00e4che im 2. Obergeschoss kann nicht direkt berechnet werden, ohne die genauen Abmessungen der Fenster zu kennen. Allerdings kann die Funktion calculate_m2_for_asset_components verwendet werden, um die Gesamtfl\\u00e4che der Fenster in diesem Bereich zu berechnen.", "sql_query": "SELECT * FROM asset_components WHERE floor = \'2. Obergeschoss\' AND type = \'IfcWindow\'"}', 'name': 'ResponseFormatter'}, 'type': 'function'}, {'id': 'c7223e0cb', 'function': {'arguments': '{"asset_component_type": "IfcWindow", "floor": "2. Obergeschoss"}', 'name': 'calculate_m2_for_asset_components'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 407, 'total_tokens': 528, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_4a36a9b9a8b8ef701032', 'i

In [5]:
model_with_tools = llm.bind_tools([ResponseFormatter])

In [6]:
model_with_tools.invoke("How wide are windows in the Facility #3?")

AIMessage(content="I don't have specific information about the width of windows in Facility #3, as there is no context provided about what Facility #3 refers to. If you could provide more details or clarify the location or purpose of Facility #3, I might be able to offer more relevant information or guidance on where to find the answer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 278, 'total_tokens': 348, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_c1b1963fdfa747d6bac1', 'id': 'chatcmpl-10635a3a-091e-4a1b-b3ed-ccdf251e6536', 'finish_reason': 'stop', 'logprobs': None}, id='run-525042da-7154-4107-a501-b7a0eef857aa-0', usage_metadata={'input_tokens': 278, 'output_tokens': 70, 'total_tokens': 348, 'input_token_details': {}, 'output_token_details': {}})